In [2]:
from reader_utils.corpus_builder_utils import *
from note_utils.pitch_dictionary import PitchDictionary
from reader_utils.dataset_builder import DatasetBuilder
from model_utils.model_utils import *
from gensim.models import Word2Vec

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
 
import seaborn as sns
sns.set_style("darkgrid")

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

Init Plugin
Init Graph Optimizer
Init Kernel


In [3]:
pd = PitchDictionary("dataset_objects/pitches_dict.txt")
db = DatasetBuilder("dataset_objects/full_corpus")

In [4]:
raw_corpus_path = "dataset_objects/raw_full_corpus"
raw_corpus = db.read_corpus_from_path(raw_corpus_path)

In [5]:
model = Word2Vec(vector_size=50, window=5, min_count=1, negative=5, epochs=10)

In [6]:
model.build_vocab(raw_corpus)

In [7]:
model.train(raw_corpus, total_examples=model.corpus_count, epochs=10, report_delay=1)

(1964392, 8810820)

In [13]:
# most_similar(positive=["woman", "king"], negative=["man"]) => 
# (king – man) + woman = ?    =>    (king – man) = ? - woman
model.wv.most_similar(positive=["do5", "sol4"], negative=["re5"], topn=3)

[('mi4', 0.5570495128631592),
 ('re4', 0.4805028438568115),
 ('fa4#4', 0.426617830991745)]